In [4]:
import cv2
import json
import os
from pycocotools.coco import COCO

# COCO annotation 파일 로드
coco = COCO('/data/ephemeral/home/Lv2.Object_Detection/test_dir/2/train.json')

# output 파일 저장 경로 설정
save_dir = '/data/ephemeral/home/Lv2.Object_Detection/level2-objectdetection-cv-12/res/output'
train2k_dir = os.path.join(save_dir, 'train2k')
os.makedirs(train2k_dir, exist_ok=True)  # train2k 디렉토리 생성

# 카테고리 정보 복사
categories = coco.loadCats(coco.getCatIds())

# 새로운 annotation을 저장할 리스트
new_annotations = []
new_images = []

# annotation 파일 불러오기
with open('/data/ephemeral/home/Lv2.Object_Detection/test_dir/2/train.json', 'r') as f:
    coco_data = json.load(f)

# annotation ID와 이미지 ID 추적
annotation_id = max([ann['id'] for ann in coco_data['annotations']]) + 1
image_id = max([img['id'] for img in coco_data['images']]) + 1

# 업샘플링된 이미지가 저장될 디렉토리
save_dir = '/data/ephemeral/home/Lv2.Object_Detection/level2-objectdetection-cv-12/res/output'
os.makedirs(save_dir, exist_ok=True)  # 폴더가 없는 경우 생성

for image_info in coco_data['images']:
    image_id_str = str(image_info['id']).zfill(4) + '.jpg'
    image_path = os.path.join('/data/ephemeral/home/Lv2.Object_Detection/test_dir/2/train', image_id_str)
    
    # 이미지 읽기
    image = cv2.imread(image_path)
    if image is None:
        print(f"이미지 {image_path}를 읽을 수 없습니다.")
        continue
    
    # 이미지 2배 업샘플링
    upsampled_image = cv2.resize(image, (image.shape[1] * 2, image.shape[0] * 2), interpolation=cv2.INTER_LINEAR)

    # 이미지 나누기 (4등분)
    height, width = upsampled_image.shape[:2]
    split_images = {
        "top_left": (0, height // 2, 0, width // 2),
        "top_right": (0, height // 2, width // 2, width),
        "bottom_left": (height // 2, height, 0, width // 2),
        "bottom_right": (height // 2, height, width // 2, width),
    }

    for split_name, (y1, y2, x1, x2) in split_images.items():
        split_img = upsampled_image[y1:y2, x1:x2]
        
        # 이미지 파일 이름 수정
        original_file_name = os.path.splitext(image_info['file_name'])[0]
        original_file_name = original_file_name[-4:]
        split_image_filename = f"{split_name}_{original_file_name}.jpg"

        # 나눈 이미지 저장 경로
        split_image_path = os.path.join(train2k_dir, split_image_filename)
        
        # 이미지를 저장하고 성공 여부를 체크
        success = cv2.imwrite(split_image_path, split_img)
        if not success:
            print(f"이미지 {split_image_path} 저장에 실패했습니다.")
        else:
            print(f"이미지 {split_image_path}가 성공적으로 저장되었습니다.")

        # 이미지 정보 저장 (새로운 이미지 ID로)
        new_image_info = {
            'id': image_id,
            'file_name': os.path.join('train2k', split_image_filename),  # 파일 이름 업데이트
            'height': y2 - y1,
            'width': x2 - x1
        }
        new_images.append(new_image_info)

        # 해당 이미지의 annotation 업데이트
        annotation_ids = coco.getAnnIds(imgIds=image_info['id'])
        annotations = coco.loadAnns(annotation_ids)

        for ann in annotations:
            # 기존 바운딩 박스를 분할된 이미지 내에서 조정
            new_bbox = [
                ann['bbox'][0] * 2 - x1,  # x
                ann['bbox'][1] * 2 - y1,  # y
                ann['bbox'][2] * 2,       # width
                ann['bbox'][3] * 2        # height
            ]
            
            # 바운딩 박스가 분할된 이미지 영역에 있는지 확인
            if (new_bbox[0] + new_bbox[2] <= 0 or new_bbox[1] + new_bbox[3] <= 0 or
                new_bbox[0] >= x2 - x1 or new_bbox[1] >= y2 - y1):
                continue  # 이미지 영역 밖이면 스킵

            # 바운딩 박스 좌표를 이미지 영역 내로 클리핑
            new_bbox[0] = max(new_bbox[0], 0)
            new_bbox[1] = max(new_bbox[1], 0)
            new_bbox[2] = min(new_bbox[2], (x2 - x1) - new_bbox[0])
            new_bbox[3] = min(new_bbox[3], (y2 - y1) - new_bbox[1])

            # 바운딩 박스가 유효한지 확인
            if new_bbox[2] <= 0 or new_bbox[3] <= 0:
                continue

            new_ann = {
                'id': annotation_id,
                'image_id': image_id,  # 새로운 이미지 ID
                'category_id': ann['category_id'],
                'bbox': new_bbox,
                'area': new_bbox[2] * new_bbox[3],
                'iscrowd': ann['iscrowd']
            }

            new_annotations.append(new_ann)
            annotation_id += 1
        
        # 이미지 ID 증가
        image_id += 1

# 새로운 annotation 파일 작성
new_coco_data = {
    'images': new_images,
    'annotations': new_annotations,
    'categories': categories
}

# 새로운 JSON 파일로 저장
with open(output_annotations_file, 'w') as f:
    json.dump(new_coco_data, f)

print(f"새로운 annotation 파일이 {output_annotations_file}에 저장되었습니다.")

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
이미지 /data/ephemeral/home/Lv2.Object_Detection/level2-objectdetection-cv-12/res/output/train2k/top_left_0000.jpg가 성공적으로 저장되었습니다.
이미지 /data/ephemeral/home/Lv2.Object_Detection/level2-objectdetection-cv-12/res/output/train2k/top_right_0000.jpg가 성공적으로 저장되었습니다.
이미지 /data/ephemeral/home/Lv2.Object_Detection/level2-objectdetection-cv-12/res/output/train2k/bottom_left_0000.jpg가 성공적으로 저장되었습니다.
이미지 /data/ephemeral/home/Lv2.Object_Detection/level2-objectdetection-cv-12/res/output/train2k/bottom_right_0000.jpg가 성공적으로 저장되었습니다.
이미지 /data/ephemeral/home/Lv2.Object_Detection/level2-objectdetection-cv-12/res/output/train2k/top_left_0003.jpg가 성공적으로 저장되었습니다.
이미지 /data/ephemeral/home/Lv2.Object_Detection/level2-objectdetection-cv-12/res/output/train2k/top_right_0003.jpg가 성공적으로 저장되었습니다.
이미지 /data/ephemeral/home/Lv2.Object_Detection/level2-objectdetection-cv-12/res/output/train2k/bottom_left_0003.jpg가 성공적으로 저장되었습니다.
이미지 /data

In [7]:
import json
import cv2
import matplotlib.pyplot as plt

# 새롭게 만든 annotation 파일 로드
with open('/data/ephemeral/home/Lv2.Object_Detection/level2-objectdetection-cv-12/res/train2kCrop.json', 'r') as f:
    new_coco_data = json.load(f)

#'/data/ephemeral/home/Lv2.Object_Detection/test_dir/2/train.json')
# 이미지 ID 설정 (예시로 첫 번째 이미지를 사용)
image_id = new_coco_data['images'][0]['id']
image_info = next(img for img in new_coco_data['images'] if img['id'] == image_id)
image_path = f"/data/ephemeral/home/Lv2.Object_Detection/test_dir/2/{image_info['file_name']}"

# 이미지 읽기
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # OpenCV는 BGR로 이미지를 읽으므로 RGB로 변환

# 해당 이미지의 annotation 정보 가져오기
annotations = [ann for ann in new_coco_data['annotations'] if ann['image_id'] == image_id]

# Bounding Box 그리기
for ann in annotations:
    bbox = ann['bbox']  # [x, y, width, height]
    x, y, w, h = bbox
    cv2.rectangle(image, (int(x), int(y)), (int(x + w), int(y + h)), (0, 255, 0), 2)  # 초록색으로 박스 그리기

# 이미지 출력 (matplotlib 사용)
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.axis('off')
plt.show()


[ WARN:0@1233.697] global loadsave.cpp:244 findDecoder imread_('/data/ephemeral/home/Lv2.Object_Detection/test_dir/2/train2k/top_left_0000.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
